# IMPORTS

In [7]:
import yaml
import numpy as np
import firebase_admin
import pandas as pd

import datetime

In [2]:
from oauth2client import file,client, tools

In [3]:
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [4]:
#GSHEETs 
# code labe for bigginers - https://codelabs.developers.google.com/codelabs/gsuite-apis-intro/#7
import httplib2
import os, io
import argparse
import auth

import unidecode

from httplib2 import Http

from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials

# CONNECT TO FIREBASE

In [5]:
filename='/Users/daniel/OAN/credentials/oan-nikarit-firebase-adminsdk-p9lf1-e140012c2e.json'

In [8]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## TEST

In [9]:
col_query = db.collection('Accounting').where(u'project', u'==', u'opunMdqGyXDlfQ9DYIbN')

In [10]:
cuentas_auara = col_query.stream()

In [12]:
#for acc in cuentas_auara:
    #print(u'{} => {}'.format(acc.id, acc.to_dict()))

## ABRIR VENTAS

In [112]:
ventas = db.collection(
                       'Nikarit_Sales'
                      ).where(u'origin', u'==', u'Shopify').stream()

In [115]:
l_ventas = []
for venta in ventas:
    d_venta = venta.to_dict()
    if d_venta['creation_date'][0:4]=='2020':
        l_ventas.append(d_venta)

In [120]:
len(l_ventas)*2+432

1190

In [117]:
cuentas = {'paypal':'vJbbj1kPxkcdXJyBOf1l',
 'shopify_payments':'iA9Pzv2CImjItzwCaQv0'}

In [118]:
d_comisiones = {'paypal':0.019,
 'shopify_payments':0.021}
d_tasa = {'paypal':0.35,
 'shopify_payments':0.25}

In [119]:
i=433
l_ventas2=[]
for venta in l_ventas:
    d_venta = venta
    d_conta={
       "creation_date":"2020-08-26T18:02:55.591Z",
        "execution_date":d_venta['creation_date'][:-6]+'.000Z',
       "place":'spain',
        "project":'0DmODGTOEiM5lg9SGx0J',
        "intervention":'w04441aFcU5b7pQm6Rd2',
        "phase":None,
       "creator_user":"z5m936GA0t3vHM28QKhR",
        "user_in_charge":"IjBxXuBshlfq2MUzwBSK",
        "type":'income',
       "amount":d_venta['total_price'],
        "vat":round((d_venta['total_price']-4)*(1-1/1.21),2),
        "account_id":'3deqBmrZ1iJkAwMmxSOp',
        "target_id":cuentas[d_venta['gateway']],
       "concept":'venta shopify',
        "description":'venta online: '+str(d_venta['order_id']),
        "images":[],
        "code":i,
        "origin":'script_dani'
        
    }
    comision=round(
                    (d_venta['total_price']*d_comisiones[d_venta['gateway']]
                    )+d_tasa[d_venta['gateway']],
                    2)
    d_comision={
       "creation_date":"2020-08-26T18:02:55.591Z",
        "execution_date":d_venta['creation_date'][:-6]+'.000Z',
       "place":'spain',
        "project":'0DmODGTOEiM5lg9SGx0J',
        "intervention":'w04441aFcU5b7pQm6Rd2',
        "phase":None,
       "creator_user":"z5m936GA0t3vHM28QKhR",
        "user_in_charge":"IjBxXuBshlfq2MUzwBSK",
        "type":'expense',
       "amount":comision,
        "vat":0,
        "account_id":'3deqBmrZ1iJkAwMmxSOp',
        "target_id":cuentas[d_venta['gateway']],
       "concept":'comision '+d_venta['gateway'],
        "description":'comision de venta: '+str(d_venta['order_id']),
        "images":[],
        "code":i+1,
        "origin":'script_dani'
        
    }
    i=i+2
    
    doc_acc = db.collection(u'Accounting').add(d_conta)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    doc_acc = db.collection(u'Accounting').add(d_comision)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    #l_ventas2.append(d_conta)
    #l_ventas2.append(d_comision)

In [80]:
l_ventas2[len(l_ventas2)-1]

{'creation_date': '2020-08-26T18:02:55.591Z',
 'execution_date': '2020-05-08T13:41:43.000Z',
 'place': 'spain',
 'project': '0DmODGTOEiM5lg9SGx0J',
 'intervention': 'w04441aFcU5b7pQm6Rd2',
 'phase': None,
 'creator_user': 'z5m936GA0t3vHM28QKhR',
 'user_in_charge': 'IjBxXuBshlfq2MUzwBSK',
 'type': 'income',
 'amount': 17,
 'vat': 2.26,
 'account_id': '3deqBmrZ1iJkAwMmxSOp',
 'target_id': 'iA9Pzv2CImjItzwCaQv0',
 'concept': 'venta shopify',
 'description': 'venta online: 2330434044003',
 'images': [],
 'code': 1650,
 'origin': 'script_dani'}

In [83]:
2*len(l_ventas)+432

1652

## VERIFICAR

In [ ]:
account_total = db.collection(
                       'Accounting'
                      ).where(u'origin', u'==', u'script_dani').stream()

# ELIMINAR

In [121]:
ventas_dani = db.collection(
                       'Accounting'
                      ).stream()

In [123]:
l_ventas = []
for venta in ventas_dani:
    l_ventas.append(venta.to_dict())

In [124]:
len(l_ventas)

1115

In [108]:
#l_ventas = []
#for venta in ventas_danis:
#    #l_ventas.append(venta.to_dict())
#    d_venta=venta.to_dict()
#    db.collection(u'Accounting').document(d_venta['id']).delete()

In [110]:
l_ventas = []
for venta in ventas_dani:
    l_ventas.append(venta.to_dict())

In [101]:
db.collection(u'Accounting').document('zwlvkHssBuygzyzGuoMA').delete()

seconds: 1598472978
nanos: 897772000

In [100]:
l_ventas[0]['id']

'06DwrZ0s8WZ7XsYAHFXj'

In [111]:
len(l_ventas)

0

# CREATE ACCOUNT

In [207]:
#Saca las cuentas 629
col_query = db.collection('Accounts').stream()

l_accounts=[]
for us in col_query:
    l_accounts.append(us.to_dict())
accnts = dict()
for us in l_accounts:
    if(str(us['number'])[0:3] == "629"):
        accnts.update({us['number']:us['name']})

In [208]:
accnts

{629101: 'WEB y Hosting',
 629001: 'Material de Oficina',
 629106: 'Gastos Telefónicos',
 629102: 'Papelería e impresiones',
 629002: 'Gasolina',
 629004: 'Servicio de gestoría',
 629005: 'Análisis (Ej. Manteca)'}

In [210]:
cuenta={
    "db_creation_date": "2020-05-27T15:09:17.217220",
    "name":"Dietas (comida)",
    "number":629008,
    "transactions_id":[],
    "type":"expense",
    "user_id":None,
    "value":0
}

In [211]:
doc_acc = db.collection(u'Accounts').add(cuenta)[1]
doc_acc_id = doc_acc.id
doc_acc.update({'id':doc_acc_id})

update_time {
  seconds: 1590612517
  nanos: 206982000
}